In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [7]:
# Problem parameters
# (Double-check with Ragusa on appropriate parameters)
sigma_s = 0 # Macroscopic cross section for scattering [cm^(-1)]
sigma_t = 1 # Macroscopic total interaction cross section [cm^(-1)] (reciprocal is neutron mfp)
q_ext = 3 
q = q_ext/2.0
# Vacuum boundary conditions
psi_left = 0
psi_right = 0

# Number of points in domain
npts = 21
# Left- and right-endpoints of domain
a = 0
b = 1
points = np.linspace(a, b, npts)
dx     = np.abs(points[1] - points[0])

# Quadrature order (needs to be odd to avoid mu = 0)
n = 16
mu_s, weights = scipy.special.roots_legendre(n)
# By default, mu_s entries go from negative to positive, and we want the opposite
mu_s = np.flip(mu_s)

# Array to store angular flux values (each row corresponds to a different angle,
# and there as many columns as there are points)
psi_vals = np.zeros((mu_s.size, npts))
# Impose BCs
psi_vals[0:int((n/2)), 0]  = psi_left # Impose left BC for forward sweep
psi_vals[int((n/2)):, -1] = psi_right # Impose right BC for reverse sweep

for mu_idx, mu_d in enumerate(mu_s):
    # If angle is greater than 0, perform a forward sweep (left -> right)
    if mu_d > 0:
        # Loop through points (cell boundaries)
        for pt_idx in np.arange(npts):
            # print(f"Forward sweep: {cell_idx}")
            # Point to the right of current point       
            if pt_idx < (npts - 1):
                pt_r_idx = pt_idx + 1 
                psi_vals[mu_idx, pt_r_idx] = (q*dx + mu_d*psi_vals[mu_idx, pt_idx] - (sigma_t/2.0)*psi_vals[mu_idx, pt_idx]*dx) / \
                                            (mu_d + ((sigma_t * dx)/2.0))
    # If angle is less than 0, perform a reverse sweep (right -> left)            
    else:
        for pt_idx in np.arange(npts-1, -1, -1):
                # Reverse sweep    
                pt_l_idx = pt_idx - 1
                psi_vals[mu_idx, pt_l_idx] = (q*dx - mu_d*psi_vals[mu_idx, pt_idx] - (sigma_t/2.0)*psi_vals[mu_idx, pt_idx]*dx) / \
                                            (-mu_d + ((sigma_t * dx)/2.0))

In [9]:
# Compute average flux (average of each consecutive 2 angular flux values for each angle)
average_flux = psi_vals[:, 0:(npts-1)] + psi_vals[:, 1:npts]/2.0
# print(average_flux[0:int(n/2), :])
print(psi_vals[int(n/2):, :])

[[1.49996873 1.4999464  1.49990813 1.49984252 1.49973005 1.49953727
  1.49920681 1.49864034 1.49766933 1.49600486 1.4931517  1.48826094
  1.47987739 1.46550667 1.44087297 1.39864693 1.32626482 1.20219043
  0.98950732 0.62493485 1.49998176]
 [1.4573616  1.44905338 1.43912627 1.42726483 1.41309216 1.3961579
  1.37592394 1.35174733 1.32285984 1.28834353 1.2471016  1.19782357
  1.13894355 1.06859057 0.98452911 0.884088   0.76407563 0.62067848
  0.44933994 0.24461556 1.46431494]
 [1.33136745 1.31189562 1.2901754  1.26594718 1.23892134 1.20877485
  1.17514739 1.13763699 1.09579531 1.04912221 0.99705982 0.93898583
  0.87420611 0.80194633 0.7213428  0.63143206 0.53113941 0.41926608
  0.29447485 0.15527411 1.34882363]
 [1.20295333 1.17790201 1.150738   1.12128311 1.08934416 1.05471166
  1.01715843 0.97643816 0.93228377 0.88440563 0.8324897  0.77619546
  0.71515368 0.64896395 0.57719214 0.49936748 0.4149795  0.32347469
  0.22425287 0.1166632  1.22605628]
 [1.10100754 1.07369442 1.04451157 1.0133